In [1]:
from math import sqrt
from statistics import mean

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling as pp
import itertools
import copy
from functools import reduce

/Users/samueleceol/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("CRISP/CSV/db/farming_system.csv")
df

,Farming system,Macro region,Agro-ecological zone,Total area (m ha),Cultivated area (m ha),Cattle population (m head),Irrigated area (m ha),Total population (m),Agricultural population (m),Farm size (ha),Description,Irrigated/rainfed
0,Lowland Rice,EAP,Humid and Moist Sub-humid,197,71,52,33,825,474,197,The lowland rice farming system is found in bo...,Both
1,Temperate Mixed,EAP,Dry Sub-humid,99,,,,,,6.5,Temperate mixed farming system is found in moi...,Both
2,Upland Intensive Mixed,EAP,Humid and Sub-humid,314,,,,,,,The upland intensive mixed agricultural system...,Rainfed
3,Coastal plantation and mixed,LAC,,186,,,,,,,Coastal plantation and mixed farming in the La...,Rainfed
4,Dryland mixed,LAC,Dry-moist Sub-humid,127,,,,,,3.5,The Dryland Mixed Farming System includes two ...,Rainfed
5,Forest based Livestock,LAC,,600,,,,,,,The forest based agricultural system is locate...,Rainfed
6,High Altitude Mixed (C. Andes),LAC,Semi-arid Dry Sub-humid,121,,,,,,3,The maize-beans agricultural system covers pre...,Rainfed
7,Irrigated,LAC,Arid and Semi-arid,200,,,,,,,The irrigated agricultural system covers large...,Irrigated
8,Maize-beans (Mesoamerica),LAC,Dry-moist Sub-humid,65,,,,,,2.55,The maize-beans agricultural system covers pre...,Rainfed
9,Dryland mixed,MENA,Semi-arid Sub-humid,42,3.84,2.59,,,,7,The dryland mixed agricultural system of the M...,Rainfed


In [3]:
def PLI_single(df, col_idx):
    res = {}
    for val in df.iloc[:, col_idx].unique():
        pli = (df[df.iloc[:, col_idx] == val].index.tolist())
        if len(pli) > 1:
            res[list(df.iloc[:, col_idx].unique()).index(val)] = pli
    return res

In [4]:
def PLI_intersect(p1, p2):
    hash = {}
    for group in p1:
        for row in p1[group]:
            hash[row] = group
    
    pli = {}
    for group in p2:
        for row in p2[group]:
            if(row in hash):
                key = [hash[row], group]

                if(tuple(key) not in pli):
                    pli[tuple(key)] = [row]
                else:
                    pli[tuple(key)] = pli[tuple(key)] + [row]
    
    return {k: v for k, v in pli.items() if len(v) > 1}

PLI_intersect({4: [4, 9], 9: [10, 17], 10: [11, 12]}, {1: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14]})

{(10, 1): [11, 12]}

In [5]:
def multi_column_PLI_intersect(df, col_idxs):
    pli = PLI_single(df, col_idxs[0])

    for i in range(1, len(col_idxs)):
        pli = PLI_intersect(pli, PLI_single(df, col_idxs[i]))

    return pli

In [6]:
def key_error(pli):
    key_error = 0
    for key in pli:
        key_error += len(pli[key]) - 1
    return key_error

In [7]:
def UCC_discovery(df):
    k = 0
    E = []
    F = []
    E.insert(k, [[[col_idx], PLI_single(df, col_idx)] for col_idx in range(0, len(df.columns))])
    F.insert(k, UCC_prune(E[k]))

    while len(F[k]) != 0:
        E.insert(k+1, UCC_candidates(F[k]))
        F.insert(k+1, UCC_prune(E[k+1]))
        k += 1

    uccs = []
    for j in range(0, k):
        ucc_level = [e[0] for e in E[j] if e not in F[j]]
        for col_comb in ucc_level:
            uccs.append([list(df.columns)[idx] for idx in col_comb])
    return uccs

def UCC_prune(E):
    F = []
    for e in E:
        if e[1]: F.append([e[0], e[1]])
    return F

def UCC_candidates(F):
    E = []
    for f_1, p_1 in F:
        for f_2, p_2 in F:
            # Column combination differs only by one and
            # Last column idx for f_1 < Last column idx for f_2
            if((f_1[:-1] == f_2[:-1]) and (f_1[-1] < f_2[-1])):
                f = f_1 + f_2[-1:]
                
                # Check subsets of f of size len(f) - 1
                if(all(item in list(zip(*F))[0] for item in [list(t) for t in itertools.combinations(f, len(f)-1)])):
                    p = PLI_intersect(p_1, p_2)
                    E.append([f, p])
    return E


In [8]:
UCC_discovery(df)

[['Total area (m ha)'],
 ['Farming system', 'Macro region'],
 ['Farming system', 'Agro-ecological zone'],
 ['Farming system', 'Farm size (ha)'],
 ['Farming system', 'Description'],
 ['Agro-ecological zone', 'Description'],
 ['Farm size (ha)', 'Description']]

In [98]:
#13_12
def FD_discovery(df):
    k = 0
    E = []
    C = []
    F = []
 
    primary_keys = [] # TODO: REMOVE

    E.insert(k, [[[col_idx], PLI_single(df, col_idx)] for col_idx in range(0, len(df.columns))])
    C.insert(k, {str(tuple([col_idx])): list(range(0,len(df.columns))) for col_idx in range(0, len(df.columns))})
    F.insert(k, FD_prune(E[k], C[k], primary_keys))

    while len(F[k]) != 0:
        E.insert(k+1, FD_candidates(F[k]))
        C.insert(k+1, FD_dependencies(E[k+1], C[k], df))
        F.insert(k+1, FD_prune(E[k+1], C[k+1], primary_keys))
        k = k+1

def FD_prune(E, C_plus, primary_keys):
    F = copy.deepcopy(E)
    for e in E:
        X = e[0]
        p = e[1]

        if(not C_plus[str(tuple(X))]): 
            F.remove([X, p])
        if(not p): # Check superkey
            primary_keys.append (X) # TODO: REMOVE
            for A in [c for c in C_plus[str(tuple(X))] if c not in X]:
                aug_lhs_list = []
                for B in X: aug_lhs_list.append(sorted([x for x in X if x != B] + [A]))

                # FIX for early pruning of primary keys
                to_be_removed = []
                for subset in aug_lhs_list: 
                    for pk in primary_keys:
                        if all(x in subset for x in pk):
                            to_be_removed.append(subset)
                aug_lhs_list = [subset for subset in aug_lhs_list if subset not in to_be_removed]
                # END FIX

                if(all(str(tuple(aug_lhs)) in C_plus for aug_lhs in aug_lhs_list)):
                    if(all(A in C_plus[str(tuple(aug_lhs))] for aug_lhs in aug_lhs_list)): 
                        print(f"{X} -> {A}")
            F.remove([X, p])
    return F

def FD_candidates(F):
    E = []
    for f_1, p_1 in F:
        for f_2, p_2 in F:
            # Column combination differs only by one and
            # Last column idx for f_1 < Last column idx for f_2
            if((f_1[:-1] == f_2[:-1]) and (f_1[-1] < f_2[-1])):
                f = f_1 + f_2[-1:]
                
                # Check subsets of f of size len(f) - 1
                if(all(item in list(zip(*F))[0] for item in [list(t) for t in itertools.combinations(f, len(f)-1)])):
                    p = PLI_intersect(p_2, p_1)
                    E.append([f, p])
    return E

def FD_dependencies(E, C, df):
    C_plus = {}
    for X, p in E:
        prev_rhs = ()
        tuple_key = ()
        for B in X: 
            prev_rhs += tuple([x for x in X if x != B])
            tuple_key += tuple([B])
        #C_plus[str(tuple_key)] = set.intersection(*map(set,[C[str(tuple([B]))] for B in X]))
        #C_plus[str(tuple_key)] = list(reduce(np.intersect1d, [C[str(tuple([B]))] for B in X]))
        C_plus[str(tuple_key)] = prev_rhs

    for X, p in E:
        for A in X:
            subset = [x for x in X if x != A]
            subset_error = key_error(multi_column_PLI_intersect(df, subset)) # TODO: pass E_k insteas ofm
            
            if(subset_error == key_error(p)): 
                print(f"{subset} -> {A}")
                C_plus[str(tuple(X))] = list(set(C_plus[str(tuple(X))]) - set([A]))

                for B in set(range(0, len(df.columns))) - set(X):
                    C_plus[str(tuple(X))] = list(set(C_plus[str(tuple(X))]) - set([B]))
    
    return C_plus

FD_discovery(df)

[3] -> 0
[3] -> 1
[3] -> 2
[3] -> 4
[3] -> 5
[3] -> 6
[3] -> 7
[3] -> 8
[3] -> 9
[3] -> 10
[3] -> 11
[0] -> 11
[10] -> 1
[2] -> 6
[2] -> 7
[2] -> 8
[4] -> 5
[4] -> 6
[4] -> 7
[4] -> 8
[10] -> 4
[10] -> 5
[7] -> 6
[6] -> 7
[8] -> 6
[6] -> 8
[10] -> 6
[8] -> 7
[7] -> 8
[10] -> 7
[10] -> 8
[10] -> 11
[0, 5] -> 4
[0, 4] -> 5
[0, 4] -> 6
[0, 4] -> 7
[0, 4] -> 8
[0, 4] -> 11
[0, 5] -> 6
[0, 5] -> 7
[0, 5] -> 8
[0, 5] -> 11
[0, 6] -> 11
[0, 7] -> 11
[0, 8] -> 11
[1, 2] -> 6
[1, 2] -> 7
[1, 2] -> 8
[2, 9] -> 1
[1, 4] -> 5
[1, 4] -> 6
[1, 4] -> 7
[1, 4] -> 8
[4, 10] -> 1
[1, 10] -> 4
[5, 10] -> 1
[1, 10] -> 5
[6, 10] -> 1
[1, 10] -> 6
[7, 10] -> 1
[1, 10] -> 7
[8, 10] -> 1
[1, 10] -> 8
[10, 11] -> 1
[1, 10] -> 11
[2, 5] -> 4
[2, 4] -> 5
[2, 4] -> 6
[2, 4] -> 7
[2, 4] -> 8
[2, 9] -> 4
[2, 5] -> 6
[2, 5] -> 7
[2, 5] -> 8
[2, 9] -> 5
[2, 9] -> 6
[2, 11] -> 6
[2, 9] -> 7
[2, 11] -> 7
[2, 9] -> 8
[2, 11] -> 8
[2, 9] -> 11
[5, 6] -> 4
[4, 6] -> 5
[4, 5] -> 6
[5, 7] -> 4
[4, 7] -> 5
[4, 5] -> 7
[5, 8]